### Comparison of optimizers for LASSO parameter range experiments

sklearn has 2 ways to fit logistic regression models with a LASSO penalty: [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) (using the liblinear optimizer; i.e. coordinate descent), and [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) (which uses stochastic gradient descent).

Here, we want to compare mutation prediction results between the two optimizers, across all the cancer genes in our driver gene set.

In [1]:
import os
import itertools as it

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
import seaborn as sns

import pancancer_evaluation.config as cfg
import pancancer_evaluation.utilities.data_utilities as du
import pancancer_evaluation.utilities.analysis_utilities as au

%load_ext autoreload
%autoreload 2

In [2]:
ll_base_results_dir = os.path.join(
    cfg.repo_root, '02_cancer_type_classification', 'results', 'lasso_range_lr_all_features'
)

# this doesn't have a sex covariate but it's probably close enough
sgd_base_results_dir = os.path.join(
    cfg.repo_root, '02_cancer_type_classification', 'results', 'lasso_range_valid'
)

training_dataset = 'all_other_cancers'
ll_results_dir = os.path.join(ll_base_results_dir, training_dataset)
sgd_results_dir = os.path.join(sgd_base_results_dir, training_dataset)

metric = 'aupr'

output_plots = False
output_plots_dir = None

### Get coefficient information for each lasso penalty

In [3]:
# these are generated from results files pretty slowly so it helps to cache them
ll_coefs_df_file = './ll_coefficients_df.tsv'

if os.path.exists(ll_coefs_df_file):
    print('exists')
    ll_nz_coefs_df = pd.read_csv(ll_coefs_df_file, sep='\t', index_col=0)
else:
    print('not exists')
    ll_nz_coefs_df = []
    # get coefficient info for training dataset specified above
    for coef_info in au.generate_nonzero_coefficients_lasso_range(ll_results_dir,
                                                                  gene='TP53'):
        (gene,
         cancer_type,
         seed,
         lasso_param,
         coefs_list) = coef_info
        for fold_no, coefs in enumerate(coefs_list):
            ll_nz_coefs_df.append(
                [gene, cancer_type, lasso_param, seed, fold_no, len(coefs)]
            )
    ll_nz_coefs_df = pd.DataFrame(
        ll_nz_coefs_df,
        columns=['gene', 'cancer_type', 'lasso_param', 'seed', 'fold', 'nz_coefs']
    )
    ll_nz_coefs_df.lasso_param = ll_nz_coefs_df.lasso_param.astype(float)
    ll_nz_coefs_df.to_csv(ll_coefs_df_file, sep='\t')
                                                                  
ll_nz_coefs_df.head()

exists


,gene,cancer_type,lasso_param,seed,fold,nz_coefs
0,TP53,READ,10.0,1,0,7851
1,TP53,READ,10.0,1,1,7711
2,TP53,READ,10.0,1,2,8009
3,TP53,READ,10.0,1,3,7716
4,TP53,GBM,1000.0,1,0,15297


In [4]:
# these are generated from results files pretty slowly so it helps to cache them
sgd_coefs_df_file = './sgd_coefficients_df.tsv'

if os.path.exists(sgd_coefs_df_file):
    print('exists')
    sgd_nz_coefs_df = pd.read_csv(sgd_coefs_df_file, sep='\t', index_col=0)
else:
    print('not exists')
    sgd_nz_coefs_df = []
    # get coefficient info for training dataset specified above
    for coef_info in au.generate_nonzero_coefficients_lasso_range(sgd_results_dir,
                                                                  gene='TP53'):
        (gene,
         cancer_type,
         seed,
         lasso_param,
         coefs_list) = coef_info
        for fold_no, coefs in enumerate(coefs_list):
            sgd_nz_coefs_df.append(
                [gene, cancer_type, lasso_param, seed, fold_no, len(coefs)]
            )
    sgd_nz_coefs_df = pd.DataFrame(
        sgd_nz_coefs_df,
        columns=['gene', 'cancer_type', 'lasso_param', 'seed', 'fold', 'nz_coefs']
    )
    sgd_nz_coefs_df.lasso_param = sgd_nz_coefs_df.lasso_param.astype(float)
    sgd_nz_coefs_df.to_csv(sgd_coefs_df_file, sep='\t')
                                                                  
sgd_nz_coefs_df.head()

exists


,gene,cancer_type,lasso_param,seed,fold,nz_coefs
0,TP53,UCEC,0.00025,42,0,6882
1,TP53,UCEC,0.00025,42,1,6827
2,TP53,UCEC,0.00025,42,2,6923
3,TP53,UCEC,0.00025,42,3,6898
4,TP53,LUAD,0.00050,1,0,6143


### Get performance information for each lasso penalty

In [5]:
# load performance information
ll_perf_df_file = './ll_perf_df.tsv'

if os.path.exists(ll_perf_df_file):
    print('exists')
    ll_perf_df = pd.read_csv(ll_perf_df_file, sep='\t', index_col=0)
else:
    print('not exists')
    ll_perf_df = au.load_prediction_results_lasso_range(ll_results_dir,
                                                        'liblinear',
                                                        gene='TP53')
    ll_perf_df.rename(columns={'experiment': 'optimizer'}, inplace=True)
    ll_perf_df.lasso_param = ll_perf_df.lasso_param.astype(float)
    ll_perf_df.to_csv(ll_perf_df_file, sep='\t')

ll_perf_df.head()

exists


,auroc,aupr,gene,holdout_cancer_type,signal,seed,data_type,fold,optimizer,lasso_param
0,1.00000,1.00000,TP53,PRAD,signal,1,train,0,liblinear,1500.0
1,0.72444,0.23601,TP53,PRAD,signal,1,test,0,liblinear,1500.0
2,0.90611,0.89081,TP53,PRAD,signal,1,cv,0,liblinear,1500.0
3,1.00000,1.00000,TP53,PRAD,signal,1,train,1,liblinear,1500.0
4,0.73958,0.37246,TP53,PRAD,signal,1,test,1,liblinear,1500.0


In [6]:
# add nonzero coefficient count
ll_plot_df = (
    ll_perf_df[(ll_perf_df.signal == 'signal')]
      .merge(ll_nz_coefs_df, left_on=['holdout_cancer_type', 'lasso_param', 'seed', 'fold'],
             right_on=['cancer_type', 'lasso_param', 'seed', 'fold'])
      .drop(columns=['cancer_type'])
      .sort_values(by=['holdout_cancer_type', 'lasso_param'])
      .reset_index(drop=True)
)
ll_plot_df.lasso_param = ll_plot_df.lasso_param.astype(float)

print(ll_plot_df.shape)
ll_plot_df.head()

(7680, 12)


,auroc,aupr,gene_x,holdout_cancer_type,signal,seed,data_type,fold,optimizer,lasso_param,gene_y,nz_coefs
0,0.5,0.47534,TP53,BLCA,signal,1,train,0,liblinear,0.001,TP53,0
1,0.5,0.54082,TP53,BLCA,signal,1,test,0,liblinear,0.001,TP53,0
2,0.5,0.47334,TP53,BLCA,signal,1,cv,0,liblinear,0.001,TP53,0
3,0.5,0.47109,TP53,BLCA,signal,1,train,1,liblinear,0.001,TP53,0
4,0.5,0.47959,TP53,BLCA,signal,1,test,1,liblinear,0.001,TP53,0


In [7]:
# load performance information
sgd_perf_df_file = './sgd_perf_df.tsv'

if os.path.exists(sgd_perf_df_file):
    print('exists')
    sgd_perf_df = pd.read_csv(sgd_perf_df_file, sep='\t', index_col=0)
else:
    print('not exists')
    sgd_perf_df = au.load_prediction_results_lasso_range(sgd_results_dir,
                                                        'liblinear',
                                                        gene='TP53')
    sgd_perf_df.rename(columns={'experiment': 'optimizer'}, inplace=True)
    sgd_perf_df.lasso_param = sgd_perf_df.lasso_param.astype(float)
    sgd_perf_df.to_csv(sgd_perf_df_file, sep='\t')

sgd_perf_df.head()

exists


,auroc,aupr,gene,holdout_cancer_type,signal,seed,data_type,fold,optimizer,lasso_param
0,0.661710,0.63834,TP53,OV,signal,42,train,0,liblinear,0.1
1,0.089431,0.84457,TP53,OV,signal,42,test,0,liblinear,0.1
2,0.668130,0.60945,TP53,OV,signal,42,cv,0,liblinear,0.1
3,0.685860,0.62514,TP53,OV,signal,42,train,1,liblinear,0.1
4,0.666670,0.97639,TP53,OV,signal,42,test,1,liblinear,0.1


In [8]:
# add nonzero coefficient count
sgd_plot_df = (
    sgd_perf_df[(sgd_perf_df.signal == 'signal')]
      .merge(sgd_nz_coefs_df, left_on=['holdout_cancer_type', 'lasso_param', 'seed', 'fold'],
             right_on=['cancer_type', 'lasso_param', 'seed', 'fold'])
      .drop(columns=['cancer_type'])
      .sort_values(by=['holdout_cancer_type', 'lasso_param'])
      .reset_index(drop=True)
)
sgd_plot_df.lasso_param = sgd_plot_df.lasso_param.astype(float)

print(sgd_plot_df.shape)
sgd_plot_df.head()

(4800, 12)


,auroc,aupr,gene_x,holdout_cancer_type,signal,seed,data_type,fold,optimizer,lasso_param,gene_y,nz_coefs
0,0.99994,0.99993,TP53,BLCA,signal,1,train,0,liblinear,0.0001,TP53,7408
1,0.86038,0.89194,TP53,BLCA,signal,1,test,0,liblinear,0.0001,TP53,7408
2,0.89676,0.87061,TP53,BLCA,signal,1,cv,0,liblinear,0.0001,TP53,7408
3,0.99978,0.99982,TP53,BLCA,signal,1,train,1,liblinear,0.0001,TP53,7397
4,0.86191,0.85000,TP53,BLCA,signal,1,test,1,liblinear,0.0001,TP53,7397


### Compare feature selection with performance